Our group plans to investigate patterns in job applications and hiring in the city of New Orleans. We plan to focus predominantly on New Orleans City Service Departments in our analysis. 
	

The first dataset we have found describes hiring data for city jobs in New Orleans. The dataset contains information about the job title/department, as well as the term length of the hire. One of the trends that we hope to further explore is a relationship between hire length and type of job. We believe that probationary hires may be unfairly exploited based on the easily terminable nature of their hiring terms.
	

This dataset has a few limitations. Firstly, it is not very large - if we hope to draw any widely applicable conclusions, we will need to find a bigger source of data. However, we believe that the trends which emerge in this dataset will help point us in the right direction for the rest of our project. Additionally, it lacks some demographic statistics that might generate more powerful conclusions - the pay of each job, the race and gender of the applicant, etc.
	

Another dataset we are considering working with is city job applications. This dataset doesn’t give us much - simply the title of the job applied for and the time of application - but it is considerably larger than the first dataset. We can at least determine which jobs are most frequently applied to at different times of year using this dataset.
	

For our collaboration plan, we are primarily using a combination of Github and Zoom meetings. We both have experience using Github to coordinate code in group projects, so we should be able to maintain an organized work environment. We have been meeting over Zoom on Sundays to touch base about work scheduling, but we are both able to have relatively flexible meeting patterns as the semester progresses.